In [1]:
import pandas as pd
import numpy as np
import sklearn
import plotly.express as px
import matplotlib.pyplot as plt
import os
from PIL import Image
import numpy as np
import random
import sys

current_dir = os.getcwd()

parent_dir = os.path.dirname(current_dir)

if parent_dir not in sys.path:
    sys.path.append(parent_dir)

from preprocessing import DATA_ROOT, DATA_FOLDERS, DATA_SUBFOLDERS, SEGMENTATION_FILES, SEGMENTATION_NAMES 

from preprocessing import load_labels, image_generator, image_path_generator, batch_generator, load_segmentation, merge_dicts


# Should look like _______ MATH4570-final-project-1\Preprocessing
# unzip the contents of the OneDrive folder into the parent directory of the git repo.
# so the 3 files should be renders, dcms (empty), hemhorrage-labels
print(os.getcwd())

C:\Users\johnn\MATH4570-final-project-1


In [2]:
df = pd.read_csv("../hemorrhage-labels.csv", index_col="Image")
labels_dict = load_labels("../Hemorrhage Segmentation Project")

In [3]:
labels_dict["ID_ffff82e46"]

{'any': 0,
 'epidural': 0,
 'intraparenchymal': 0,
 'intraventricular': 0,
 'subarachnoid': 0,
 'subdural': 0}

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten, MaxPooling2D
from tensorflow.keras.layers import Conv2D 

NUM_LABELS = 6

model = Sequential()
model.add(Conv2D(32, (11,11),
                 padding='valid',
                 input_shape=(512, 512,3),
                 activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64, (7,7),
                 activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
 
model.add(Flatten())
model.add(Dense(units=128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=NUM_LABELS, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.summary()

C:\Users\johnn\AppData\Roaming\Python\Python39\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 502, 502, 32)   │        11,648 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 251, 251, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 245, 245, 64)   │       100,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 122, 122, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 122, 122, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 952576)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │   121,929,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 6)              │           774 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 122,042,694 (465.56 MB)

 Trainable params: 122,042,694 (465.56 MB)

 Non-trainable params: 0 (0.00 B)

Now we train the model on batches, we treat each batch as an epoch and we train on 9/10 of the batch, testing on the final 1/10

In [ ]:
batch_size = 200
epi_batch_gen = batch_generator('C:/Users/johnn/Downloads/XN_Project/XN_Project/renders/epidural/epidural/max_contrast_window', 
                            batch_size=batch_size)
intraparenchymal_batch_gen = batch_generator('C:/Users/johnn/Downloads/XN_Project/XN_Project/renders/intraparenchymal/intraparenchymal/max_contrast_window', 
                            batch_size=batch_size)
intraventricular_batch_gen = batch_generator('C:/Users/johnn/Downloads/XN_Project/XN_Project/renders/intraventricular/intraventricular/max_contrast_window', 
                            batch_size=batch_size)
subarachnoid_batch_gen = batch_generator('C:/Users/johnn/Downloads/XN_Project/XN_Project/renders/subarachnoid/subarachnoid/max_contrast_window', 
                            batch_size=batch_size)
#subdural_batch_gen = batch_generator('C:/Users/johnn/Downloads/XN_Project/XN_Project/renders/subdural/subdural/max_contrast_window', 
#                            batch_size=batch_size)

epochs = 10

history = None

for epoch in range(epochs):
    epi_images, epi_ids = next(epi_batch_gen)
    intraparenchymal_images, intraparenchymal_ids = next(intraparenchymal_batch_gen)
    intraventricular_images, intraventricular_ids = next(intraventricular_batch_gen)
    subarachnoid_images, subarachnoid_ids = next(subarachnoid_batch_gen)
    #subdural_images, subdural_ids = next(subdural_batch_gen)
        
    images = np.concatenate([epi_images, intraparenchymal_images, intraventricular_images, subarachnoid_images])
    ids = np.concatenate([epi_ids, intraparenchymal_ids, intraventricular_ids, subarachnoid_ids])
    
    p = np.random.permutation(len(images))
    images = images[p]
    ids = ids[p]
    
    Y_train = np.array([])
    X_train = images[0:int(0.9*batch_size*4)]
    Y_test = np.array([])
    X_test = images[int(0.9*batch_size*4):]
    X_train = X_train / 255
    X_test = X_test / 255
    
    for i, (img, file_id) in enumerate(zip(images[0:int(0.9*batch_size*4)], ids[0:int(0.9*batch_size*4)])):
        temp = []
        if (labels_dict[file_id]['any'] == 0):
            temp = [1]
        else:
            temp = [0]
        Y_train = np.append(Y_train, np.append(temp, np.fromiter(labels_dict[file_id].values(), dtype=float)[1:]))
    
    for i, (img, file_id) in enumerate(zip(images[int(0.9*batch_size*4):], ids[int(0.9*batch_size*4):])):
        temp = []
        if (labels_dict[file_id]['any'] == 0):
            temp = [1]
        else:
            temp = [0]
        Y_test = np.append(Y_test, np.append(temp, np.fromiter(labels_dict[file_id].values(), dtype=float)[1:]))
    
    Y_train = Y_train.reshape(-1, 6)
    Y_test = Y_test.reshape(-1, 6)
    
    history = model.fit(X_train, Y_train,
          batch_size=100,
          verbose=1,
          validation_data=(X_test, Y_test))

pd.DataFrame(history.history).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 1)
 
plt.show()


8/8 ━━━━━━━━━━━━━━━━━━━━ 278s 30s/step - accuracy: 0.1674 - loss: 58.0116 - val_accuracy: 0.2750 - val_loss: 7.4899
8/8 ━━━━━━━━━━━━━━━━━━━━ 272s 31s/step - accuracy: 0.2464 - loss: 3.9456 - val_accuracy: 0.2250 - val_loss: 1.7028
8/8 ━━━━━━━━━━━━━━━━━━━━ 284s 33s/step - accuracy: 0.2584 - loss: 3.1866 - val_accuracy: 0.1875 - val_loss: 1.6867
8/8 ━━━━━━━━━━━━━━━━━━━━ 262s 30s/step - accuracy: 0.2869 - loss: 2.4658 - val_accuracy: 0.2000 - val_loss: 5.8058
8/8 ━━━━━━━━━━━━━━━━━━━━ 311s 34s/step - accuracy: 0.2491 - loss: 5.5578 - val_accuracy: 0.2500 - val_loss: 1.5644
8/8 ━━━━━━━━━━━━━━━━━━━━ 252s 29s/step - accuracy: 0.2608 - loss: 7.5967 - val_accuracy: 0.2625 - val_loss: 1.4731
8/8 ━━━━━━━━━━━━━━━━━━━━ 686s 27s/step - accuracy: 0.2874 - loss: 1.4915 - val_accuracy: 0.2750 - val_loss: 1.4486
1/8 ━━━━━━━━━━━━━━━━━━━━ 7:00 60s/step - accuracy: 0.3200 - loss: 1.4311

There are 6 classifiable hemorrhages classes (including the no hemorrhage class). Computation cost is quite high for this code so I cannot run too many epochs, but it is clear that there is an upward trend, and we are only training on a batch of size 800/750,000 cell lines. These results, given batch size and epoch count, seem very promising. With extensive training from the full dataset, I'm assuming we would see quite a strong model.

# Trying a pretrained model

We can give a pretrained model a try:

In [5]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds

batch_size = 64
epi_batch_gen = batch_generator('C:/Users/johnn/Downloads/XN_Project/XN_Project/renders/epidural/epidural/max_contrast_window', 
                            batch_size=batch_size)
intraparenchymal_batch_gen = batch_generator('C:/Users/johnn/Downloads/XN_Project/XN_Project/renders/intraparenchymal/intraparenchymal/max_contrast_window', 
                            batch_size=batch_size)
intraventricular_batch_gen = batch_generator('C:/Users/johnn/Downloads/XN_Project/XN_Project/renders/intraventricular/intraventricular/max_contrast_window', 
                            batch_size=batch_size)
subarachnoid_batch_gen = batch_generator('C:/Users/johnn/Downloads/XN_Project/XN_Project/renders/subarachnoid/subarachnoid/max_contrast_window', 
                            batch_size=batch_size)
#subdural_batch_gen = batch_generator('C:/Users/johnn/Downloads/XN_Project/XN_Project/renders/subdural/subdural/max_contrast_window', 
#                            batch_size=batch_size)

epochs = 50

def get_hem_type(label):
    if (label[0] == 1):
        return 0
    elif (label[1] == 1):
        return 1
    elif (label[2] == 1):
        return 2
    elif (label[3] == 1):
        return 3
    elif (label[4] == 1):
        return 4
    else:
        return 5

def preprocess(image, label):
    resized_image = tf.image.resize(image, [224, 224])
    final_image = keras.applications.xception.preprocess_input(resized_image)
    hem_label = get_hem_type(label)
    return final_image, hem_label

batch = 64

base_model = tf.keras.applications.xception.Xception(weights="imagenet", include_top=False)
avg = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
output = tf.keras.layers.Dense(6, activation="softmax")(avg)
pretrained_model = tf.keras.Model(inputs=base_model.input, outputs=output)

pretrained_model.summary()

for layer in base_model.layers:
    layer.trainable = False
    
optimizer = tf.keras.optimizers.SGD(learning_rate=0.2, momentum=0.9, decay=0.01)
pretrained_model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

for epoch in range(epochs):
    epi_images, epi_ids = next(epi_batch_gen)
    intraparenchymal_images, intraparenchymal_ids = next(intraparenchymal_batch_gen)
    intraventricular_images, intraventricular_ids = next(intraventricular_batch_gen)
    subarachnoid_images, subarachnoid_ids = next(subarachnoid_batch_gen)
    #subdural_images, subdural_ids = next(subdural_batch_gen)
        
    images = np.concatenate([epi_images, intraparenchymal_images, intraventricular_images, subarachnoid_images])
    ids = np.concatenate([epi_ids, intraparenchymal_ids, intraventricular_ids, subarachnoid_ids])
    
    p = np.random.permutation(len(images))
    images = images[p]
    ids = ids[p]
    
    Y_train = np.array([])
    X_train = images[0:int(0.9*batch_size*4)]
    Y_test = np.array([])
    X_test = images[int(0.9*batch_size*4):]
    X_train = X_train / 255
    X_test = X_test / 255
    
    for i, (img, file_id) in enumerate(zip(images[0:int(0.9*batch_size*4)], ids[0:int(0.9*batch_size*4)])):
        temp = []
        if (labels_dict[file_id]['any'] == 0):
            temp = [1]
        else:
            temp = [0]
        Y_train = np.append(Y_train, np.append(temp, np.fromiter(labels_dict[file_id].values(), dtype=float)[1:]))
    
    for i, (img, file_id) in enumerate(zip(images[int(0.9*batch_size*4):], ids[int(0.9*batch_size*4):])):
        temp = []
        if (labels_dict[file_id]['any'] == 0):
            temp = [1]
        else:
            temp = [0]
        Y_test = np.append(Y_test, np.append(temp, np.fromiter(labels_dict[file_id].values(), dtype=float)[1:]))
        
    Y_train = Y_train.reshape(-1, 6)
    Y_test = Y_test.reshape(-1, 6)
    
    train_dataset = tf.data.Dataset.from_tensor_slices((X_train, Y_train)).map(preprocess).batch(batch).prefetch(1)
    test_dataset = tf.data.Dataset.from_tensor_slices((X_test, Y_test)).map(preprocess).batch(batch).prefetch(1)
    
    history = pretrained_model.fit(train_dataset, epochs=1, validation_data=test_dataset)

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, None,      │          0 │ -                 │
│ (InputLayer)        │ None, 3)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv1        │ (None, None,      │        864 │ input_layer_1[0]… │
│ (Conv2D)            │ None, 32)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv1_bn     │ (None, None,      │        128 │ block1_conv1[0][… │
│ (BatchNormalizatio… │ None, 32)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv1_act    │ (None, None,      │          0 │ block1_conv1_bn[… │
│ (Activation)        │ None, 32)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv2        │ (None, None,      │     18,432 │ block1_conv1_act… │
│ (Conv2D)            │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv2_bn     │ (None, None,      │        256 │ block1_conv2[0][… │
│ (BatchNormalizatio… │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv2_act    │ (None, None,      │          0 │ block1_conv2_bn[… │
│ (Activation)        │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_sepconv1     │ (None, None,      │      8,768 │ block1_conv2_act… │
│ (SeparableConv2D)   │ None, 128)        │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_sepconv1_bn  │ (None, None,      │        512 │ block2_sepconv1[… │
│ (BatchNormalizatio… │ None, 128)        │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_sepconv2_act │ (None, None,      │          0 │ block2_sepconv1_… │
│ (Activation)        │ None, 128)        │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_sepconv2     │ (None, None,      │     17,536 │ block2_sepconv2_… │
│ (SeparableConv2D)   │ None, 128)        │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_sepconv2_bn  │ (None, None,      │        512 │ block2_sepconv2[… │
│ (BatchNormalizatio… │ None, 128)        │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, None,      │      8,192 │ block1_conv2_act… │
│                     │ None, 128)        │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_pool         │ (None, None,      │          0 │ block2_sepconv2_… │
│ (MaxPooling2D)      │ None, 128)        │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, None,      │        512 │ conv2d_4[0][0]    │
│ (BatchNormalizatio… │ None, 128)        │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_12 (Add)        │ (None, None,      │          0 │ block2_pool[0][0… │
│                     │ None, 128)        │            │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block3_sepconv1_act │ (None, None,      │          0 │ add_12[0][0]    

 Total params: 20,873,774 (79.63 MB)

 Trainable params: 20,819,246 (79.42 MB)

 Non-trainable params: 54,528 (213.00 KB)

4/4 ━━━━━━━━━━━━━━━━━━━━ 18s 4s/step - accuracy: 0.2576 - loss: 1.6465 - val_accuracy: 0.0769 - val_loss: 1.7655
4/4 ━━━━━━━━━━━━━━━━━━━━ 14s 3s/step - accuracy: 0.2161 - loss: 1.4851 - val_accuracy: 0.1538 - val_loss: 1.4511
4/4 ━━━━━━━━━━━━━━━━━━━━ 13s 3s/step - accuracy: 0.2245 - loss: 1.4689 - val_accuracy: 0.2308 - val_loss: 1.3806
4/4 ━━━━━━━━━━━━━━━━━━━━ 12s 3s/step - accuracy: 0.2418 - loss: 1.4261 - val_accuracy: 0.2692 - val_loss: 1.4025
4/4 ━━━━━━━━━━━━━━━━━━━━ 13s 3s/step - accuracy: 0.2276 - loss: 1.4458 - val_accuracy: 0.2692 - val_loss: 1.7140
4/4 ━━━━━━━━━━━━━━━━━━━━ 12s 3s/step - accuracy: 0.2705 - loss: 1.6145 - val_accuracy: 0.2692 - val_loss: 1.5979
4/4 ━━━━━━━━━━━━━━━━━━━━ 13s 3s/step - accuracy: 0.2462 - loss: 1.6269 - val_accuracy: 0.3077 - val_loss: 1.4358
4/4 ━━━━━━━━━━━━━━━━━━━━ 13s 3s/step - accuracy: 0.2271 - loss: 1.4935 - val_accuracy: 0.1538 - val_loss: 1.4708
4/4 ━━━━━━━━━━━━━━━━━━━━ 13s 3s/step - accuracy: 0.2887 - loss: 1.4515 - val_accuracy: 0.1154 - 

KeyboardInterrupt: 